In [10]:
import pandas as pd
from river import naive_bayes, tree, metrics, drift
from sklearn.model_selection import train_test_split

In [11]:
def get_data():
    # Placeholder for your actual data loading mechanism
    df = pd.read_csv("dataverse/rt_8873985678962563_abrupto.csv")
    return df[['X1', 'X2']], df['class']

In [12]:
# Load the dataset
X, y = get_data()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Initialize drift detectors
drift_detectors = {
    'ADWIN': drift.ADWIN(),
    'KSWIN': drift.KSWIN(),
    'DDM': drift.binary.DDM(),
    'EDDM': drift.binary.EDDM(),
    'PageHinkley': drift.PageHinkley()
}

# Initialize base learners
base_learners = {
    'Naive Bayes': naive_bayes.GaussianNB(),
    'Hoeffding Tree': tree.HoeffdingTreeClassifier()
}

# Initialize metrics
metrics_dict = {name: {'accuracy': metrics.Accuracy(), 'auc': metrics.ROCAUC()} for name in base_learners}

In [14]:
# Function to train and test models with different drift detectors
def train_and_evaluate(base_learner_name, base_learner, drift_detector):
    model = base_learner.clone()
    model_metrics = {metric_name: metric.clone() for metric_name, metric in metrics_dict[base_learner_name].items()}
    i = 0
    
    for x, y_true in zip(X_train.to_dict(orient='records'), y_train):
        pred = model.predict_one(x)
        is_correct = int(pred != y_true)
        
        # Update drift detector with the error signal (0 or 1)
        in_drift = drift_detector.update(is_correct).change if hasattr(drift_detector.update(is_correct), 'change') else False
        
        if in_drift:
            print(f"Change detected at index {i}")
            model = base_learner.clone()
            model_metrics = {metric_name: metric.clone() for metric_name, metric in metrics_dict[base_learner_name].items()}
        
        model.learn_one(x, y_true)
        
#         for metric in model_metrics.values():
#             metric.update(float(y_true), float(pred))
        i = i + 1
    
    for x, y_true in zip(X_test.to_dict(orient='records'), y_test):
        pred = model.predict_one(x)
        for metric in model_metrics.values():
            metric.update(y_true, pred)
    
    return model_metrics

In [15]:

# Evaluate each combination of base learner and drift detector
for drift_name, drift_detector in drift_detectors.items():
    for learner_name, learner in base_learners.items():
        print(f"Evaluating {learner_name} with {drift_name}:")
        evaluated_metrics = train_and_evaluate(learner_name, learner, drift_detector.clone())
        for metric_name, metric in evaluated_metrics.items():
            print(f"{metric_name}: {metric.get()}")



Evaluating Naive Bayes with ADWIN:
accuracy: 0.655
auc: 0.6518166422474463
Evaluating Hoeffding Tree with ADWIN:
accuracy: 0.6869166666666666
auc: 0.6863718631151413
Evaluating Naive Bayes with KSWIN:
accuracy: 0.655
auc: 0.6518166422474463
Evaluating Hoeffding Tree with KSWIN:
accuracy: 0.6869166666666666
auc: 0.6863718631151413
Evaluating Naive Bayes with DDM:
accuracy: 0.655
auc: 0.6518166422474463
Evaluating Hoeffding Tree with DDM:
accuracy: 0.6869166666666666
auc: 0.6863718631151413
Evaluating Naive Bayes with EDDM:
accuracy: 0.655
auc: 0.6518166422474463
Evaluating Hoeffding Tree with EDDM:
accuracy: 0.6869166666666666
auc: 0.6863718631151413
Evaluating Naive Bayes with PageHinkley:
accuracy: 0.655
auc: 0.6518166422474463
Evaluating Hoeffding Tree with PageHinkley:
accuracy: 0.6869166666666666
auc: 0.6863718631151413
